In [1]:
osm = '/home/jasp/osm2lulc/osm_terceira.xml'

refraster = '/home/jasp/osm2lulc/lmt_terceira.shp'

result = '/mnt/d/osm2lulc2023/osm2lulc_v2/madeira_lulc.shp'

epsg = 5015

In [2]:
import os
import datetime as dt

from glass.sql.db           import create_db
from glass.it.db import osm_to_psql
from glass.pys.oss import mkdir, fprop
from glass.pys.tm import now_as_str
from glass.cons.osmtolulc import osm_features
from glass.gp.gen.sql import st_dissolve
from glass.prop.sql import row_num

In [3]:
ws = mkdir(os.path.join(
    os.path.dirname(osm),
    'az_' + now_as_str()
), overwrite=True)

In [4]:
polygons = 'multipolygons'
lines = 'lines'

### Pre-processamento

In [5]:
osm_db = create_db(fprop(osm, 'fn', forceLower=True), overwrite=True)
osm_db = osm_to_psql(osm, osm_db)

### Get OSM Features and classes

In [6]:
osmfeat = osm_features()

osmfeat['whr'] = osmfeat['table'] + '.' + \
    osmfeat['key'] + "='" + osmfeat['value'] + "'"

### Module 1

In [6]:
m1feat = osmfeat[(osmfeat.table == 'multipolygons') & (osmfeat.module == 1)]

time_a = dt.datetime.now().replace(microsecond=0)

polygons = 'multipolygons'
for cls in m1feat.cos2022.unique():
    df_cls = m1feat[m1feat.cos2022 == cls]
    
    whr_str = df_cls.whr.str.cat(sep=' OR ')
    
    nrow = row_num(
        osm_db, polygons,
        where=whr_str, api='psql'
    )
    
    if not nrow: continue
    
    shp = st_dissolve(
        osm_db, polygons,
        f'ST_Transform(wkb_geometry, {epsg})',
        os.path.join(ws, f'mod1_{str(cls)}.shp'),
        whrClause=whr_str,
        outTblIsFile=True,
        valascol={'lulc' : cls},
        geomout='geom', api='psql',
        multipart=None
    )

time_b = dt.datetime.now().replace(microsecond=0)

print(time_b - time_a)

/home/jasp/.virtualenvs/genv/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


0:00:13


### Module 2

In [7]:
m2feat = osmfeat[
    (osmfeat.table == 'lines') &
    (osmfeat.module == 2) &
    (osmfeat.distbf > 0)
]

m2feat['caseaux'] = 'WHEN ' + m2feat['key'] + '=\'' + \
    m2feat['value'] + '\' THEN ' + m2feat['distbf']

time_a = dt.datetime.now().replace(microsecond=0)

nbuild = cnt_rows(
    osm_db, polygons,
    where="building IS NOT NULL",
    api='psql'
)

caselulc = 

time_b = dt.datetime.now().replace(microsecond=0)

print(time_b - time_a)

In [8]:
m2feat

,id,key,value,table,module,cos2022,alternative,distbf
102,103,highway,lines.highway='motorway',lines,2,1411,-1,12
103,104,highway,lines.highway='trunk',lines,2,1411,-1,12
104,105,highway,lines.highway='primary',lines,2,1411,-1,12
105,106,highway,lines.highway='secondary',lines,2,1411,-1,12
106,107,highway,lines.highway='tertiary',lines,2,1411,-1,12
107,108,highway,lines.highway='unclassified',lines,2,1411,-1,6
108,109,highway,lines.highway='residential',lines,2,1411,-1,6
109,110,highway,lines.highway='service',lines,2,1411,-1,6
111,112,highway,lines.highway='motorway_link',lines,2,1411,-1,6
112,113,highway,lines.highway='trunk_link',lines,2,1411,-1,6
